## import necessary libraries for pruning

In [2]:
import torch, sys, os
import torch.nn as nn
import logging, torch.onnx
import matplotlib.pyplot as plt


from mask_mobile_net import MobileNetV2
from cnn.resNet.utils import calculate_cost
from cnn.resNet.resnet_example import get_data_loaders
from thop import profile
from torch import optim

#### asuming model size

In [4]:

def calculate_block_contributions(model, input_size=(2, 3, 32, 32)):
    # Get the device of the model
    device = next(model.parameters()).device
    # Create a dummy input tensor
    dummy_input = torch.randn(input_size).to(device)

    # Profile initial layers (conv1, bn1, relu)
    initial_layers = nn.Sequential(model.conv1, model.bn1, model.relu)
    macs_initial, _ = profile(initial_layers, inputs=(dummy_input,), verbose=False)
    params_initial = sum(p.numel() for p in initial_layers.parameters())

    # Pass input through initial layers
    x = initial_layers(dummy_input)

    # Define and profile each block sequentially
    blocks = [model.block1, model.block2, model.block3, model.block4,
              model.block5, model.block6, model.block7]
    macs_blocks = []
    params_blocks = []

    for block in blocks:
        macs, _ = profile(block, inputs=(x,), verbose=False)
        macs_blocks.append(macs)
        params = sum(p.numel() for p in block.parameters())
        params_blocks.append(params)
        x = block(x)  # Update the input for the next block

    # Profile final layers (conv2, bn2, avgpool, fc)
    final_layers = nn.Sequential(model.conv2, model.bn2, model.avgpool, nn.Flatten(start_dim=1), model.fc)
    macs_final, _ = profile(final_layers, inputs=(x,), verbose=False)
    params_final = sum(p.numel() for p in final_layers.parameters())

    # Combine initial and final contributions into fixed parts
    macs_fixed = macs_initial + macs_final
    params_fixed = params_initial + params_final
    print(f"macs_blocks: {macs_blocks}")
    print(f"params_blocks: {params_blocks}")
    print(f"macs_fixed: {macs_fixed}")
    print(f"params_fixed: {params_fixed}")
    return macs_blocks, params_blocks, macs_fixed, params_fixed

#### Training model

In [5]:
def train_with_constraints(model, train_loader, criterion, optimizer, device,
                          lambda_macs, lambda_size, macs_blocks, params_blocks,
                          macs_fixed, params_fixed):
    """Train the model with penalties to enforce MACs and size constraints."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for data in train_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        classification_loss = criterion(outputs, labels)

        # Compute expected MACs and size
        mask_weights = torch.sigmoid(model.mask)
        expected_macs = macs_fixed + sum(mask_weights[i] * macs_blocks[i] for i in range(7))
        expected_size = (params_fixed + sum(mask_weights[i] * params_blocks[i] for i in range(7))) * 4 / 1e6  # MB

        # Penalties for exceeding thresholds

        macs_penalty = torch.relu(expected_macs - model.mac_threshold)
        size_penalty = torch.relu(expected_size - model.size_threshold)

        macs_penalty = macs_penalty.to(device)
        size_penalty = size_penalty.to(device)


        #print(f"classification_loss device: {classification_loss.device}")

        # Total loss
        loss = classification_loss + lambda_macs * macs_penalty + lambda_size * size_penalty
        #print(f"Before backward: mask = {model.mask.data}")
        loss.backward()
        #print(f"After backward: mask = {model.mask.data}")

        optimizer.step()
        #print(f"After optimizer step: mask = {model.mask.data}")
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

### plotting function

In [6]:

# Plotting function
def plot_metrics(epochs, losses, accuracies, macs_list, sizes_list):
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    axs[0, 0].plot(range(1, epochs + 1), losses, label='Loss')
    axs[0, 0].set_title('Training Loss')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()

    axs[0, 1].plot(range(1, epochs + 1), accuracies, label='Accuracy', color='orange')
    axs[0, 1].set_title('Training Accuracy')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('Accuracy (%)')
    axs[0, 1].legend()

    axs[1, 0].plot(range(1, epochs + 1), macs_list, label='MACs', color='green')
    axs[1, 0].set_title('Model MACs')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('MACs')
    axs[1, 0].legend()

    axs[1, 1].plot(range(1, epochs + 1), sizes_list, label='Size', color='red')
    axs[1, 1].set_title('Model Size (MB)')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('Size (MB)')
    axs[1, 1].legend()

    plt.tight_layout()
    plt.savefig('output/mobile_net_soft_pruning_metrices.png')
    plt.show()

#### Main fucntion

In [7]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize model with thresholds
    size_threshold = 2.0  # MB
    mac_threshold = 10000   # Example MACs threshold (adjust as needed)
    # todo: accuracy threshold should be implemented here
    model = MobileNetV2(num_classes=10, size_threshold=size_threshold, mac_threshold=mac_threshold).to(device)


    #torch.save(model.state_dict(), f"output/before_train_mobile_net_v2.pth")
    #convert_to_onnx(model, f"output/onnx/before_train_mobile_net_v2.onnx")

    # for name, param in model.named_parameters():
    #     print(f"name: {name}: {param.device}")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss().to(device)

    # for group in optimizer.param_groups:
    #     for p in group['params']:
    #         state = optimizer.state[p]
    #         if state:
    #             print(f"Parameter {p.device}, exp_avg: {state['exp_avg'].device}")



    train_loader, _ = get_data_loaders('./data')

    # Precompute MACs and parameter contributions
    macs_blocks, params_blocks, macs_fixed, params_fixed = calculate_block_contributions(model)

    # Hyperparameters for penalties
    lambda_macs = 1e-4  # Adjust based on scale of MACs
    lambda_size = 1e-2  # Adjust based on scale of size

    # Log initial architecture
    print("Before Training Architecture:", model.get_network_description())
    initial_macs = macs_fixed + sum(torch.sigmoid(model.mask[i]) * macs_blocks[i] for i in range(7))
    initial_size = (params_fixed + sum(torch.sigmoid(model.mask[i]) * params_blocks[i] for i in range(7))) * 4 / 1e6
    print(f"Initial MACs: {initial_macs:.2e}, Initial Size: {initial_size:.2f} MB")

    # Metric tracking lists
    losses = []
    accuracies = []
    macs_list = []
    sizes_list = []

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        loss, acc = train_with_constraints(model, train_loader, criterion, optimizer, device, lambda_macs, lambda_size, macs_blocks, params_blocks, macs_fixed, params_fixed)

        # Compute current MACs and size for logging
        mask_weights = torch.sigmoid(model.mask)
        current_macs = macs_fixed + sum(mask_weights[i] * macs_blocks[i] for i in range(7))
        current_size = (params_fixed + sum(mask_weights[i] * params_blocks[i] for i in range(7))) * 4 / 1e6

        # Store metrics
        # Append metrics, ensuring they are CPU scalars
        losses.append(loss.item() if isinstance(loss, torch.Tensor) else loss)
        accuracies.append(acc.item() if isinstance(acc, torch.Tensor) else acc)
        macs_list.append(current_macs.cpu().item() if isinstance(current_macs, torch.Tensor) else current_macs)
        sizes_list.append(current_size.cpu().item() if isinstance(current_size, torch.Tensor) else current_size)

        logging.info(f"Epoch {epoch+1}: Loss: {loss:.4f}, Accuracy: {acc:.2f}%, "
                     f"MACs: {current_macs:.2e}, Size: {current_size:.2f} MB")
        logging.info(f"Network: {model.get_network_description()}")

        # todo: implement stopping criteria here
        # todo: currently stopped imidiately after the condition is met without any further training
        # Check if thresholds are met
        if current_macs <= mac_threshold and current_size <= size_threshold:
            logging.info("Thresholds satisfied!")
            continue


    torch.save(model.state_dict(), f"output/structural_prune_after_train_mobile_net_v2.pth")
    #convert_to_onnx(model, f"output/onnx/after_train_mobile_net_v2.onnx")
    # Log final architecture
    final_mask_weights = torch.sigmoid(model.mask)
    final_macs = macs_fixed + sum(final_mask_weights[i] * macs_blocks[i] for i in range(7))
    final_size = (params_fixed + sum(final_mask_weights[i] * params_blocks[i] for i in range(7))) * 4 / 1e6
    print(f"Final Architecture: {model.get_network_description()}")
    print(f"Final MACs: {final_macs:.2e}, Final Size: {final_size:.2f} MB")
    # Plot the metrics
    plot_metrics(len(losses), losses, accuracies, macs_list, sizes_list)
    """
    example_input = torch.randn(1, 3, 32, 32).to(device)
    onnx_path = f"output/onnx/structural_prune_after_train_mobile_net_v2.onnx"

    torch.onnx.export(
        model,
        example_input,
        onnx_path,
        export_params=True,
        opset_version=12,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    print(f"✅ Model saved as ONNX to {onnx_path}")
    model.eval()
    with torch.no_grad():
        output = model(example_input)
        print(output.shape)
    """


In [ ]:
if __name__ == "__main__":
    main()

## Structural Pruning

### Seed Network

In [5]:
import torch
from torch import nn

class InvertedResidual(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, expansion=6):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        self.use_res_connect = self.stride == 1 and in_channels == out_channels

        # Standard PyTorch layers (NO torch_pruning wrappers needed)
        self.conv1 = nn.Conv2d(in_channels, in_channels * expansion, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels * expansion)
        self.relu = nn.ReLU6(inplace=True)

        self.conv2 = nn.Conv2d(
            in_channels * expansion, in_channels * expansion, kernel_size=3,
            stride=stride, padding=1, groups=in_channels * expansion, bias=False
        )
        self.bn2 = nn.BatchNorm2d(in_channels * expansion)

        self.conv3 = nn.Conv2d(in_channels * expansion, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        identity = x if self.use_res_connect else None

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.use_res_connect:
            return identity + out
        else:
            return out

### Mask Network

In [2]:
from torch import nn


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10):
        super(MobileNetV2, self).__init__()
        # --- Remove mask-related parameters ---
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU6(inplace=True)

        # Define blocks (no mask_index or mask)
        self.block1 = InvertedResidual(32, 16, stride=1)
        self.block2 = InvertedResidual(16, 24, stride=2)
        self.block3 = InvertedResidual(24, 32, stride=2)
        self.block4 = InvertedResidual(32, 64, stride=2)
        self.block5 = InvertedResidual(64, 96, stride=1)
        self.block6 = InvertedResidual(96, 160, stride=2)
        self.block7 = InvertedResidual(160, 320, stride=1)

        self.conv2 = nn.Conv2d(320, 1280, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(1280)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        # --- Remove mask-based block skipping ---
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

### Save model as ONNX

In [9]:
def save_model_as_onnx(model, example_input, output_path):
    # Set model to evaluation mode
    model.eval()
    # Export to ONNX
    torch.onnx.export(
        model,
        example_input,
        output_path,
        export_params=True,
        opset_version=13,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
    )
    print(f"✅ Model saved as ONNX to {output_path}")

### Main function for Pruning

In [10]:
import torch
import torch.nn as nn
import torch_pruning as tp
from torch import optim
from cnn.resNet.resnet_example import get_data_loaders

def calculate_macs(model, example_input):
    macs, params = tp.utils.count_ops_and_params(model, example_input)
    return macs


def prune_model(model, example_input, target_macs):
    # 1. Define pruning ratio based on MACs
    current_macs = calculate_macs(model, example_input)
    macs_sparsity = 1 - (target_macs / current_macs)

    # todo: can Implement different pruner strategies here based on the importance scoring function,
    # todo: ref: main_imagenet.py file of a torch_pruning repo
    # 2. Initialize pruner
    pruner = tp.pruner.MagnitudePruner(
        model,
        example_input,
        importance=tp.importance.MagnitudeImportance(p=2),  # L2 norm
        ch_sparsity= 0.5, # todo: I can set different sparsity for each layer,  I will apply sparsity based on the macs and flops
        root_module_types=[nn.Conv2d, nn.Linear],  # Layers to prune
        ignored_layers=[model.fc],  # DO NOT prune the final classifier!
    )

    # 3. Prune
    pruner.step()

    return model

def print_model_metrics(model, example_input, label):
    macs = calculate_macs(model, example_input)
    params = sum(p.numel() for p in model.parameters())
    size_mb = params * 4 / 1e6  # 4 bytes per float32
    print(f"{label}: MACs={macs:.2e}, Size={size_mb:.2f} MB")

# Usage in main():


def train_model(model, train_loader, criterion, optimizer, device, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            # todo: try to set different optimizer after pruning , early stopping, scheduler, # of epochs for fine tuning, pruning
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch {epoch+1}: Loss={epoch_loss:.4f}, Accuracy={epoch_acc:.2f}%")
    return model

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MobileNetV2(num_classes=10).to(device)
    example_input = torch.randn(1, 3, 32, 32).to(device)  # CIFAR-10 input shape

    # Save initial model (before pruning)
    torch.save(model.state_dict(), "output/mobilenetv2_before_pruning.pth")
    save_model_as_onnx(model, example_input, "output/onnx/mobilenetv2_before_pruning.onnx")

    # Train and prune
    train_loader, test_loader = get_data_loaders('./data')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss().to(device)
    model = train_model(model, train_loader, criterion, optimizer, device, num_epochs=10)

    # Prune
    initial_macs = calculate_macs(model, example_input)
    target_macs = initial_macs // 2
    print_model_metrics(model, example_input, "Before Pruning")
    model = prune_model(model, example_input, target_macs)
    print_model_metrics(model, example_input, "After Pruning")
    # Save pruned model
    torch.save(model.state_dict(), "output/mobilenetv2_after_pruning.pth")
    save_model_as_onnx(model, example_input, "output/mobilenetv2_after_pruning.onnx")

    # Fine-tune and save final model
    model = train_model(model, train_loader, criterion, optimizer, device, num_epochs=10)
    torch.save(model.state_dict(), "output/mobilenetv2_final.pth")
    save_model_as_onnx(model, example_input, "output/mobilenetv2_final.onnx")

if __name__ == "__main__":
    main()

✅ Model saved as ONNX to output/mobilenetv2_before_pruning.onnx
Using dataset directory: /home/muis/thesis/github-repo/master-thesis/cnn/mobile_net_v2/data
Epoch 1: Loss=1.6700, Accuracy=38.68%
Epoch 2: Loss=1.2926, Accuracy=53.77%
Epoch 3: Loss=1.1241, Accuracy=60.28%
Epoch 4: Loss=0.9963, Accuracy=65.26%
Epoch 5: Loss=0.9004, Accuracy=68.39%
Epoch 6: Loss=0.8265, Accuracy=71.05%
Epoch 7: Loss=0.7562, Accuracy=73.67%
Epoch 8: Loss=0.7054, Accuracy=75.24%
Epoch 9: Loss=0.6582, Accuracy=77.14%
Epoch 10: Loss=0.6139, Accuracy=78.47%
Before Pruning: MACs=6.06e+06, Size=4.68 MB
After Pruning: MACs=1.84e+06, Size=1.22 MB
✅ Model saved as ONNX to output/mobilenetv2_after_pruning.onnx


/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch_pruning/pruner/algorithms/base_pruner.py:87: UserWarning: ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.
  warnings.warn(


Epoch 1: Loss=4.2010, Accuracy=20.21%
Epoch 2: Loss=4.1631, Accuracy=20.47%
Epoch 3: Loss=4.1648, Accuracy=20.54%
Epoch 4: Loss=4.1589, Accuracy=20.54%
Epoch 5: Loss=4.1633, Accuracy=20.63%
Epoch 6: Loss=4.1515, Accuracy=20.66%
Epoch 7: Loss=4.1579, Accuracy=20.66%
Epoch 8: Loss=4.1605, Accuracy=20.71%
Epoch 9: Loss=4.1630, Accuracy=20.70%
Epoch 10: Loss=4.1623, Accuracy=20.57%
✅ Model saved as ONNX to output/mobilenetv2_final.onnx


In [11]:
import netron
netron.start("output/mobilenetv2_before_pruning.onnx")  # Before pruning
netron.start("output/mobilenetv2_after_pruning.onnx")   # After pruning

Serving 'output/mobilenetv2_before_pruning.onnx' at http://localhost:8080
Serving 'output/mobilenetv2_after_pruning.onnx' at http://localhost:8081


('localhost', 8081)